## stanfordnlp/imdb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [19]:
# Dataset isim ve directory belirle !!!!!!!!!!!!!!!!!!!!!!

imdb_train_dataset_filepath = directory+"/imdb_dataset/train_imdb.csv"
imdb_test_dataset_filepath = directory+"/imdb_dataset/test_imdb.csv"

news_train_dataset_filepath = directory+"/news_dataset/train_news.csv"
news_test_dataset_filepath = directory+"/news_dataset/test_news.csv"

total_dataset = 2500

In [20]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Veri kümesini yükle
ds = load_dataset("stanfordnlp/imdb")

# 2) Label (etiket) sınıflarını elde et
label_names = ds["train"].features["label"].names
print("Label classes:", label_names)

# 3) Her bir label sınıfının sayısını yazdır
for label_id, label_name in enumerate(label_names):
    label_count = ds["train"]["label"].count(label_id)
    print(f"Label '{label_name}' count:", label_count)

# Train ve test splitlerini DataFrame'e çevir
train_df = pd.DataFrame(ds["train"])
test_df = pd.DataFrame(ds["test"])

# Train ve test verilerini birleştir
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Her bir label sınıfından eşit sayıda örnek al
# Toplam total_dataset satır istiyoruz, bu yüzden her label'dan total_dataset / label_sayısı kadar örnek alacağız
label_counts = full_df['label'].value_counts()
min_count = label_counts.min()  # En az sayıda olan label sayısı
samples_per_label = total_dataset // len(label_counts)  # Her label'dan alınacak örnek sayısı

# Eğer her label'dan daha fazla örnek almak mümkünse, o zaman bu sayıyı kullan
samples_per_label = min(samples_per_label, min_count)

# Eşit dağılım için örnek al
balanced_df = full_df.groupby('label').apply(lambda x: x.sample(samples_per_label, random_state=42)).reset_index(drop=True)

# Toplam satır sayısı (train split için)
print("Train split row count:", len(balanced_df))

# Veri setinde text ve label sütunları mevcuttur.
# %80 train, %20 test olacak şekilde dengeli ayır
train_balanced, test_balanced = train_test_split(
    balanced_df,
    test_size=0.2,
    stratify=balanced_df["label"],
    random_state=42
)

print("Train shape:", train_balanced.shape)
print("Test shape:", test_balanced.shape)

print("\nEğitim kümesindeki label dağılımı:")
print(train_balanced["label"].value_counts())

print("\nTest kümesindeki label dağılımı:")
print(test_balanced["label"].value_counts())

# Ayrılmış veriyi CSV formatında kaydet
train_balanced.to_csv(imdb_train_dataset_filepath, index=False)
test_balanced.to_csv(imdb_test_dataset_filepath, index=False)

Label classes: ['neg', 'pos']
Label 'neg' count: 12500
Label 'pos' count: 12500
Train split row count: 2500
Train shape: (2000, 2)
Test shape: (500, 2)

Eğitim kümesindeki label dağılımı:
label
1    1000
0    1000
Name: count, dtype: int64

Test kümesindeki label dağılımı:
label
0    250
1    250
Name: count, dtype: int64


<ipython-input-20-6047e4905bfb>:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = full_df.groupby('label').apply(lambda x: x.sample(samples_per_label, random_state=42)).reset_index(drop=True)


In [21]:
from datasets import Dataset, DatasetDict
import pandas as pd

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

# CSV dosyalarından veriyi pandas DataFrame'lerine aktar
train_balanced = pd.read_csv(directory+"/imdb_dataset/train_imdb.csv")
test_balanced = pd.read_csv(directory+"/imdb_dataset/test_imdb.csv")

# Pandas DataFrame'lerini Hugging Face Dataset nesnelerine dönüştür
train_ds = Dataset.from_pandas(train_balanced)
test_ds = Dataset.from_pandas(test_balanced)

# Tek bir DatasetDict nesnesinde topla
ds = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

# Veri setini kontrol edelim
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


## fancyzhx/ag_news

In [6]:
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# 1) Veri kümesini yükle
ds2 = load_dataset("fancyzhx/ag_news")

# 2) Label (etiket) sınıflarını elde et
label_names = ds2["train"].features["label"].names
print("Label classes:", label_names)

# ds içindeki train ve test split'lerini pandas DataFrame'e çevir
train_df = pd.DataFrame(ds2["train"])
test_df = pd.DataFrame(ds2["test"])

# 2) İki split'i birleştir
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Her bir label sınıfından eşit sayıda örnek al
# Toplam total_dataset satır istiyoruz, bu yüzden her label'dan total_dataset / label_sayısı kadar örnek alacağız
label_counts = full_df['label'].value_counts()
min_count = label_counts.min()  # En az sayıda olan label sayısı
samples_per_label = total_dataset // len(label_counts)  # Her label'dan alınacak örnek sayısı

# Eğer her label'dan daha fazla örnek almak mümkünse, o zaman bu sayıyı kullan
samples_per_label = min(samples_per_label, min_count)

# Eşit dağılım için örnek al
balanced_df = full_df.groupby('label').apply(lambda x: x.sample(samples_per_label, random_state=42)).reset_index(drop=True)

# Toplam satır sayısı (train split için)
print("Train split row count:", len(balanced_df))

# Veri setinde text ve label sütunları mevcuttur.
# %80 train, %20 test olacak şekilde dengeli ayır
train_balanced, test_balanced = train_test_split(
    balanced_df,
    test_size=0.2,
    stratify=balanced_df["label"],
    random_state=42
)

# 4) Oluşan eğitim ve test veri çerçevelerini (DataFrame) CSV formatında kaydet
train_balanced.to_csv(news_train_dataset_filepath, index=False)
test_balanced.to_csv(news_test_dataset_filepath, index=False)

print("Eğitim kümesi boyutu:", train_balanced.shape)
print("Test kümesi boyutu:", test_balanced.shape)
print("\nEğitim kümesindeki sınıfların dağılımı:")
print(train_balanced["label"].value_counts())
print("\nTest kümesindeki sınıfların dağılımı:")
print(test_balanced["label"].value_counts())
print("\nVeriler 'train_news.csv' ve 'test_news.csv' olarak kaydedildi.\n")



Label classes: ['World', 'Sports', 'Business', 'Sci/Tech']
Train split row count: 2500
Eğitim kümesi boyutu: (2000, 2)
Test kümesi boyutu: (500, 2)

Eğitim kümesindeki sınıfların dağılımı:
label
1    500
2    500
0    500
3    500
Name: count, dtype: int64

Test kümesindeki sınıfların dağılımı:
label
3    125
2    125
0    125
1    125
Name: count, dtype: int64

Veriler 'train_news.csv' ve 'test_news.csv' olarak kaydedildi.



<ipython-input-22-56519d6e34c1>:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = full_df.groupby('label').apply(lambda x: x.sample(samples_per_label, random_state=42)).reset_index(drop=True)


In [23]:
from datasets import Dataset, DatasetDict
import pandas as pd

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

# 5) CSV dosyalarından tekrar okuyarak Hugging Face Dataset'e dönüştürme
train_csv = pd.read_csv(directory+"/news_dataset/train_news.csv")
test_csv = pd.read_csv(directory+"/news_dataset/test_news.csv")

train_dataset = Dataset.from_pandas(train_csv)
test_dataset = Dataset.from_pandas(test_csv)

ds2 = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(ds2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})
